<a href="https://colab.research.google.com/github/zayeem00/MLProject_TeessideUni/blob/master/Neural_Network_Regression_LUSC_on_250_Features_with_kernel_initialization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error 
from matplotlib import pyplot as plt
import seaborn as sb
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBRegressor

In [172]:
from google.colab import files
files.upload()

Saving LUAD_250_NoDepth.xlsx to LUAD_250_NoDepth.xlsx
Saving survival_LUAD.xlsx to survival_LUAD.xlsx


{'LUAD_250_NoDepth.xlsx': b'PK\x03\x04\x14\x00\x00\x00\x08\x00{\x86\x85N\x1f#\xcf\x03\xc0\x00\x00\x00\x13\x02\x00\x00\x0b\x00\x00\x00_rels/.rels\xad\x92O\x8b\xc2@\x0c\xc5\xbfJ\x99\xfb\x1aW\xc1\xc3b=y\xe9mY\xfc\x02q&\xfdC;\x93!\x13\xb1~{\x87\xbdl\xb7TP\xf0\x18^\xf2\xde\x8fG\xf6?4\xa0v\x1cR\xdb\xc5T\x8c~\x08\xa94\xadj\xfc\x02H\xb6%\x8fi\xc5\x91BVj\x16\x8f\x9aGi \xa2\xed\xb1!\xd8\xac\xd7;\x90\xa9\x879\xec\xa7\x9eE\xe5J#\x95\xfb4\xc5\t\xa5!-\xcd8\xc0\x95\xa5?3\xf7\xabl\x9b\x85[\xa4gB\xb9\xae;KG\xb6\x17OA\x17\xb2g\x1b\x06\x96Y6\x7f,\x8e\xed\xb7p\xbe\xb4,\xf4\x1a\xcd\xe3\n\xc0\x93\xa2C\xc5_\xd7\x8f\x98\x03H\xb4\xa3\xf4\x08h\xbb\x00\x841\xbe\xbb\x1d\x1a\x95\x82#7#\x82\x7f?p\xb8\x03PK\x03\x04\x14\x00\x00\x00\x08\x00{\x86\x85N\'\xe8\x87\x0e\x82\x00\x00\x00\xb1\x00\x00\x00\x10\x00\x00\x00docProps/app.xmlM\x8eM\x0b\xc20\x10D\xffJ\xe9\xddn)\xe8Ab@\xb0G\xc1\x93\xf7\x90nl \xc9\x86\xcd\n\xf9\xf9\xa6\x82\x1f\xb7y\xbca\x18uc\xca\xc8\xe2\xb1t5\x86TN\xfd*\x92\x8f\x00\xc5\xae\x18M\x19\x9aN\xcd8\xe2h\xa4!?

In [0]:
import pandas as pd
tar=pd.read_excel('survival_LUAD.xlsx')
dt=pd.read_excel('LUAD_250_NoDepth.xlsx',index_col=0)

In [174]:
from sklearn.model_selection import train_test_split
X_train,X_test,Y_train,Y_test=train_test_split(dt,tar,test_size=0.3)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(361, 250) (361, 1)
(156, 250) (156, 1)


In [207]:
NN_model = Sequential()

# The Input Layer :
NN_model.add(Dense(128, kernel_initializer='normal',input_dim = X_train.shape[1], activation='relu'))

# The Hidden Layers :
NN_model.add(Dense(200, kernel_initializer='uniform',activation='relu'))
NN_model.add(Dense(200, kernel_initializer='uniform',activation='relu'))
NN_model.add(Dense(200, kernel_initializer='uniform',activation='relu'))
NN_model.add(Dense(200, kernel_initializer='uniform',activation='relu'))
NN_model.add(Dense(200, kernel_initializer='uniform',activation='relu'))
#NN_model.add(Dense(200, kernel_initializer='normal',activation='relu'))

# The Output Layer :
NN_model.add(Dense(1, kernel_initializer='normal',activation='linear'))

# Compile the network :
NN_model.compile(loss='mean_absolute_error', optimizer='adam', metrics=['mean_absolute_error'])
NN_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_231 (Dense)            (None, 128)               32128     
_________________________________________________________________
dense_232 (Dense)            (None, 200)               25800     
_________________________________________________________________
dense_233 (Dense)            (None, 200)               40200     
_________________________________________________________________
dense_234 (Dense)            (None, 200)               40200     
_________________________________________________________________
dense_235 (Dense)            (None, 200)               40200     
_________________________________________________________________
dense_236 (Dense)            (None, 200)               40200     
_________________________________________________________________
dense_237 (Dense)            (None, 1)                 201       
Total para

In [0]:
checkpoint_name = 'Weights-{epoch:03d}--{val_loss:.5f}.hdf5' 
checkpoint = ModelCheckpoint(checkpoint_name, monitor='val_loss', verbose = 1, save_best_only = True, mode ='auto')
callbacks_list = [checkpoint]

In [0]:
NN_model.fit(X_train, Y_train, epochs=100, batch_size=32, validation_split = 0.3,callbacks=callbacks_list) 

In [210]:
predictions = NN_model.predict(X_train)
from sklearn.metrics import r2_score
accuracy=r2_score(Y_train,predictions)
print('train')
print(accuracy)

train
0.2041730881804258


In [0]:
NN_model.fit(X_test, Y_test, epochs=100, batch_size=32, validation_split = 0.3,callbacks=callbacks_list) 

In [212]:
predictions = NN_model.predict(X_test)
from sklearn.metrics import r2_score
accuracy=r2_score(Y_test,predictions)
print('test')
print(accuracy)

test
0.7462993841821116


In [0]:
'''On adding more layers:
with 3 layers:
train: 0.7
test: 0.57

with  5 layers: 
  train=0.7
  test=0.9
  
with 6 layers:
  train= 0.61
   test= 0.70
   
on adding more layers(after 5 layers and activation function for hidden layers is relu; output layer= linear, input layer=relu), the scores are decreasing.'''

In [0]:
'''   on changing the activation function
  worse with tanh
  not good with sigmoid
  best with relu


'''

In [0]:
''' on changing the number of neurons:
Tissue 
the test scores increase after increasing the neurons. But after one point, with 600 neurons, the scores are not that great. even after the layers increase, the scores are not improving.
'''

In [0]:
'''
getting a better score for 300  neurons than 400 . best score with 200 neurons and 5 layers. the score decraese before and after 200 neurons.
'''

In [0]:
''' 
changing the kernel_initialzation to uniform (But the train scores are lesser than the score that we got in 'normal' kernel_initialization) , 
random_uniform: not good.
'''